In [1]:
from mip import *



In [2]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

In [3]:
instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

In [4]:
instanceBig1 = {
    "m" : 6,
    "n" : 9,
    "l" : [190, 185, 185, 190, 195, 185],
    "s" : [11, 11, 23, 16, 2, 1, 24, 14, 20],
    "distances" : [
        [0, 200, 119, 28, 180, 77, 145, 61, 123, 87],
        [200, 0, 81, 207, 38, 122, 55, 138, 76, 113],
        [119, 81, 0, 126, 69, 121, 26, 117, 91, 32],
        [28, 207, 126, 0, 187, 84, 152, 68, 130, 94],
        [170, 38, 79, 177, 0, 92, 58, 108, 46, 98],
        [77, 122, 121, 84, 102, 0, 100, 16, 46, 96],
        [145, 55, 26, 152, 58, 100, 0, 91, 70, 58],
        [61, 138, 113, 68, 118, 16, 91, 0, 62, 87],
        [123, 76, 91, 130, 56, 46, 70, 62, 0, 66],
        [87, 113, 32, 94, 94, 96, 58, 87, 66, 0]
    ]
}

In [5]:
instance = instanceBig1

In [6]:
n = instance["n"]
m = instance["m"]
s = instance["s"]
l = np.array(instance["l"])
max_distance = np.max(instance["distances"], axis=1).sum()

# deposit = n + 1

model = Model(solver_name=CBC)

# x[i, j1, j2] is 1 if the courier i travels from node j1 to node j2
x = model.add_var_tensor((m, n + 1, n + 1), var_type=BINARY, name="x")

# y[i] is the distance traveled by the courier i
y = model.add_var_tensor((m,), var_type=INTEGER, lb=0, ub=max_distance, name="y")

# t[i, j] is 1 if the courier i carries the package j
t = model.add_var_tensor((m, n), var_type=BINARY, name="t")

# z is a path counter where z[i, j] is the number of the node j in the path of the courier i
z = model.add_var_tensor((m, n + 1), var_type=INTEGER, lb=0, ub=max_distance, name="z")

# M is a sufficiently large value for the path counter
M = n + 3

# The counter is 0 for the deposit
model += z[:, n] == 0

for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n): # Note that j2 cannot be n + 1 (i.e. the deposit)
            # if x[i, j1, j2] = 1, then z[i, j2] = z[i, j1] + 1

            model += z[i, j2] - z[i, j1] <= M * (1 - x[i, j1, j2]) + 1
            model += z[i, j1] - z[i, j2] <= M * (1 - x[i, j1, j2]) - 1


for i in range(m):
    model += np.sum(x[i, n, :]) == 1
    model += np.sum(x[i, :, n]) == 1


# t[i, j] is 1 if the courier i carries the package j
for i in range(m):
    for j in range(n):
        model += np.sum(x[i, j, :]) == t[i, j]

# All packages are carried by exactly one courier
for j in range(n):
    model += np.sum(t[:, j]) == 1

# If a courier arrives to a package, it has to leave from that package
for i in range(m):
    for j in range(n):
        courier_arrives = np.sum(x[i, :, j])
        courier_leaves = np.sum(x[i, j, :])
        # (not arrives) or leaves
        model += (1 - courier_arrives) + courier_leaves >= 1


"""# Each node must be a middle stop exactly once
for j in range(n):
    model += np.sum(x[i, j, :]) == 1
    model += np.sum(x[i, :, j]) == 1"""

# Each courier does not exceed its capacity
for i in range(m):
    model += (t[i, :] @ s) <= l[i]

# y[i] is the distance traveled by the courier i
for i in range(m):
    model +=  np.sum(x[i, :, :] * np.array(instance["distances"])) == y[i]

# You cannot travel from a node to itself
for i in range(m):
    for j in range(n):
        model += x[i, j, j] == 0


v = model.add_var(name="v", var_type=INTEGER, lb=0, ub=max_distance)

for i in range(m):
    model += v >= y[i]

model.threads = 16

In [7]:
print(model.threads)

16


In [8]:
model.objective = minimize(v)

model.optimize()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 114 small elements
Coin0506I Presolve 1053 (-234) rows, 601 (-120) columns and 4932 (-612) elements
Clp1000I sum of infeasibilities 6.24066e-05 - average 5.92656e-08, 206 fixed columns
Coin0506I Presolve 81 (-972) rows, 395 (-206) columns and 1530 (-3402) elements
Clp0029I End of values pass after 395 iterations
Clp0014I Perturbing problem by 0.001% of 3.3126433 - largest nonzero change 1.4452061e-05 ( 0.00053541528%) - largest zero change 2.8682824e-05
Clp0000I Optimal - objective value 50
Clp0000I Optimal - objective value 50
Coin0511I After Postsolve, objective 50, infeasibilities - dual 0 (0), primal 0 (0)
Clp0029I End of values pass after 18 iterations
Clp0000I Optimal - objective value 50
Clp0000I Optimal - objective value 50
Clp0000I Optimal - objective value 50
Coin0511I After Po

<OptimizationStatus.OPTIMAL: 0>

In [9]:
for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n + 1):
            if x[i, j1, j2].x == 1:
                print(f"Courier {i} travels from {j1} to {j2}")

Courier 0 travels from 0 to 3
Courier 0 travels from 3 to 9
Courier 0 travels from 9 to 0
Courier 1 travels from 5 to 9
Courier 1 travels from 7 to 5
Courier 1 travels from 9 to 7
Courier 2 travels from 9 to 9
Courier 3 travels from 1 to 2
Courier 3 travels from 2 to 9
Courier 3 travels from 9 to 1
Courier 4 travels from 6 to 9
Courier 4 travels from 9 to 6
Courier 5 travels from 4 to 9
Courier 5 travels from 8 to 4
Courier 5 travels from 9 to 8


In [10]:
def find_next(i, node):
    for j in range(n + 1):
        if x[i, node, j].x == 1:
            return j


for i in range(m):
    steps = [n]
    current_node = n
    
    current_node = find_next(i, current_node)
    steps.append(current_node)
    while current_node != n:
        current_node = find_next(i, current_node)
        steps.append(current_node)
    
    print(f'Courier {i}:', ', '.join([str(node) for node in steps]))


Courier 0: 9, 0, 3, 9
Courier 1: 9, 7, 5, 9
Courier 2: 9, 9
Courier 3: 9, 1, 2, 9
Courier 4: 9, 6, 9
Courier 5: 9, 8, 4, 9


In [11]:
print([y[i].x for i in range(m)])

[209.0, 199.0, 0.0, 226.0, 116.0, 220.0]


In [12]:
print(model.objective_value)

226.0


In [13]:
print([(i, j) for j in range(n) for i in range(m) if t[i, j].x == 1])

[(0, 0), (3, 1), (3, 2), (0, 3), (5, 4), (1, 5), (4, 6), (1, 7), (5, 8)]
